In [76]:
%matplotlib qt
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import interactive
import numpy as np

interactive(True)

In [77]:
logfile_folder = "logfiles/"
# input_filename = "Woss_Repeater_data_20231009_1688042028_1698402213.csv"
# input_filename = "Woss IT Data 2023-11-11.csv"
input_filename = "Shelly2_1692368875_1714382790.csv"

columns = {
    'Unix'        : 'int64',
    'Voltage_1'   : 'float32',
    'Voltage_2'   : 'float32',
    'Current'     : 'float32',
    'Power'       : 'float32',
    'Temperature' : 'float32',
    'SOC'         : 'float32',
    'Power status': 'string'
}


# Edit these values to change how the data is processed
startDate = '2021-11-01'
endDate = '2024-12-01'

moving_ave_window = 5

In [78]:
df = pd.read_csv(logfile_folder + input_filename, names=columns.keys(), sep=',', header=0, dtype=columns)
df['dateTime'] = pd.to_datetime(df['Unix'], unit='s')
columns['dateTime'] = 'string'

In [79]:
# Filter down to the requested date range
mask = (df['dateTime'] > startDate) & (df['dateTime'] <= endDate)
df = df.loc[mask]

In [80]:
movingAve_df = df.copy()

for feature_name in df.columns:
    if(feature_name == 'DateTime'):
        continue
    if(columns[feature_name] == 'string'):
        continue
    movingAve_df[feature_name] = df.rolling(window=moving_ave_window)[feature_name].mean()

# Comment out this line to not use the averaged values
df = movingAve_df

In [81]:
plt.figure()
ax = plt.gca()

df.plot(x='dateTime', y='Voltage_1', ax=ax, label='Batt Voltage')
# df.plot(x='dateTime', y='Voltage_2', ax=ax, label='Solar Voltage')

plt.title(input_filename)
plt.xlabel("Date")
plt.ylabel("Voltage")

plt.tight_layout()
plt.grid()
plt.show()

In [82]:
energy_per_step = df['dateTime'].diff() / np.timedelta64(1, 'h') * df['Power']

In [83]:
# This is needed for the following resamples to work. It breaks things if I do it earlier though. 
df.set_index('dateTime', inplace=True)

In [84]:
# df['dateTime'] = pd.to_datetime(df['dateTime'])
# max_solar_per_day = df.groupby(df.dateTime.dt.day)['Voltage_1'].max()

# resample the battery voltage and find daily max/mins
max_values_per_day = df['Voltage_1'].resample('D').max()
min_values_per_day = df['Voltage_1'].resample('D').min()

In [90]:
# energy_per_step = df['dateTime'].diff() / np.timedelta64(1, 'h') * df['Power']

energy_per_step.head(10)

# energy_per_step.insert(1, 'dateTime', df['dateTime'])
# df.insert(len(df.columns), 'energy',energy_per_step)

# energy_per_day = energy_per_step.resample('D').sum()

# df.head(10)
# energy_per_day.head(10)


0         NaN
1         NaN
2         NaN
3         NaN
4   -0.039716
5   -0.039737
6   -0.039716
7   -0.039686
8   -0.039652
9   -0.043167
dtype: float64

In [91]:

# energy_per_step.insert(1, 'dateTime', df['dateTime'])
df.insert(len(df.columns), 'energy',energy_per_step)


# energy_per_day = energy_per_step.resample('D').sum()

df.head(10)
# energy_per_day.head(10)


ValueError: cannot insert energy, already exists

In [92]:
energy_per_day = df.resample('D').sum()
energy_per_day.head(10)

,Unix,Voltage_1,Voltage_2,Current,Power,Temperature,SOC,Power status,energy
dateTime,,,,,,,,,
2023-08-18,1.479305e+13,114820.418605,103.767201,25816.061399,362892.580058,144073.20,0.0,OFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFO...,0.0
2023-08-19,4.874245e+12,36197.788429,33.330800,2822.215403,39092.176040,43813.80,0.0,OFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFO...,0.0
2023-08-20,4.874494e+12,36427.910808,33.040800,3772.158805,51975.833989,42609.25,0.0,OFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFO...,0.0
2023-08-21,4.874743e+12,36270.514804,33.021600,3805.661001,53990.916022,42781.25,0.0,OFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFO...,0.0
2023-08-22,4.874992e+12,35821.809771,32.461400,-20.752797,153.859968,43077.50,0.0,OFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFO...,0.0
2023-08-23,4.875241e+12,35465.643160,32.227600,-1108.292596,-13435.381998,41603.50,0.0,OFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFO...,0.0
2023-08-24,4.875490e+12,36632.751462,32.801000,9803.632018,136477.158207,44552.75,0.0,OFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFO...,0.0
2023-08-25,4.875738e+12,36859.013785,32.462200,8958.483217,126061.925996,52010.05,0.0,OFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFO...,0.0
2023-08-26,4.875987e+12,36747.172605,32.305000,9441.235205,132948.586065,59447.45,0.0,OFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFOFFO...,0.0


In [88]:
max_values_per_day.plot(marker='o', linestyle='-', label='maximum voltage', title='Max/Min Battery Voltage per Day')
min_values_per_day.plot(marker='o', linestyle='-', label='minimum voltage')
plt.xlabel('Date')
plt.ylabel('Battery Voltage')
plt.grid(True)
plt.legend()
plt.show()

In [89]:
# 2880 pts/day
# 1pt/30sec
solar_above_12V = df[df['Voltage_2'] > 10]
solar_minutes_above_12V_per_day = solar_above_12V.resample('D').count() / 2
# print(solar_minutes_above_12V_per_day)

# solar_minutes_above_12V_per_day['Voltage_2'].plot(label='Minutes of solar power per day')
plt.xlabel('Date')
plt.ylabel('Number of minutes of solar power')
plt.title('Number of minutes of solar power (voltage above 12V) per day')
plt.grid(True)
plt.show()